# Load Agent

Load Agent (extracts the typical devices’ loads)
Predict the device load!


We do not include an explanation for the Load Agent since we do not consider the extracted typical load profile of every shiftable device as informative to the users

## **1. Load And Preprocess Data**

This part's only purpose is to load the data used in the Load Agent. This process is described in detail in the Preparation Agent. 

**Note: When computing the script with another Household than Household 1 you might need to adapt some parameters**

### **1.1 Initialize And Load Python Scripts**

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
dir = 'D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main'
os.chdir(dir)

from helper_functions import Helper
from agents import Preparation_Agent, Activity_Agent
import pandas as pd

helper = Helper()

dbfile  = "D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main/home-assistant_Chris.db"


C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### **1.2 Set Parameters For Pre-processing Step**

In [51]:
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 1
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 1
}

aggregate_params = {
    'resample_param': '60T'
}
#update with active appliances attributes_ids
activity_params = {
    'active_appliances': [573,579,603,605],
    'threshold': .15
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

shiftable_devices = [573,579] # computer und tv sind m. E. non-shiftable, VR

device_params = {
    'threshold': 0.15
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

### **1.3 Pre-process Data For Input In Device_Usage Agent**

In [52]:
# calling the preparation pipelin
import pandas as pd
prep = Preparation_Agent(helper.export_sql2(dbfile))
output, scaled, df = prep.pipeline_load(prep.input, load_pipe_params)
df

[outlier truncation: 573]: 100%|█████████████████████████████████████████████████████████████| 651/651 [00:00<?, ?it/s]


[outlier truncation: 573]: 0 outliers were truncated.




[outlier truncation: 579]: 100%|█████████████████████████████████████████████████████████████| 302/302 [00:00<?, ?it/s]


[outlier truncation: 579]: 0 outliers were truncated.




[outlier truncation: 603]: 0it [00:00, ?it/s]


[outlier truncation: 603]: 0 outliers were truncated.




[outlier truncation: 605]: 0it [00:00, ?it/s]

[outlier truncation: 605]: 0 outliers were truncated.


[MinMaxScaler] Finished scaling the data.


attributes_id,573,579,603,605,573_usage,579_usage
last_updated,,,,,,
2022-11-21 17:00:00,22.990909,0.000000,0.0,0.0,1,0
2022-11-21 18:00:00,24.871795,0.320513,0.0,0.0,1,0
2022-11-21 19:00:00,0.000000,0.000000,0.0,0.0,0,0
2022-11-21 20:00:00,14.541727,1.000719,0.0,0.0,1,0
2022-11-21 21:00:00,9.292381,0.882857,0.0,0.0,0,0
...,...,...,...,...,...,...
2022-11-30 13:00:00,29.324176,3.995055,0.0,0.0,1,1
2022-11-30 14:00:00,26.513514,1.751351,0.0,0.0,1,0
2022-11-30 15:00:00,0.000000,0.000000,0.0,0.0,0,0


## **2.  Constructing the Load Agent**

### **2.1 Initialize Agent**

First we define the **Load Agent class**. It takes as input the data generated by the prep.pipeline_usage function computed above.

In [53]:
class Load_Agent:
    def __init__(self, load_input_df):
        self.input = load_input_df

### Truncate start and end dates if observations are missing
Truncating days where there are less than 24 reported hours

In [54]:
# selecting the correct data, identifying device runs, creating load profiles
# -------------------------------------------------------------------------------------------
def prove_start_end_date(self, df, date):
    import pandas as pd

    start_date = (df.index[0]).strftime("%Y-%m-%d")
    end_date = date

    if len(df.loc[start_date]) < 24:
        start_date = (pd.to_datetime(start_date) + pd.Timedelta(days=1)).strftime(
            "%Y-%m-%d"
        )
        df = df[start_date:end_date]
    else:
        df = df[:end_date]

    if len(df.loc[end_date]) < 24:
        end_new = (pd.to_datetime(end_date) - pd.Timedelta(days=1)).strftime(
            "%Y-%m-%d"
        )
        df = df[:end_new]
    else:
        df = df[:end_date]
    return df

# add to Load agent
setattr(Load_Agent, 'prove_start_end_date', prove_start_end_date)
del prove_start_end_date 

In [55]:
date = '2022-11-30'

In [56]:
Load_Agent_i = Load_Agent(df) 
Load_Agent_i.prove_start_end_date(df, date)

attributes_id,573,579,603,605,573_usage,579_usage
last_updated,,,,,,
2022-11-22 00:00:00,0.000000,0.0000,0.0,0.0,0,0
2022-11-22 01:00:00,0.000000,0.0000,0.0,0.0,0,0
2022-11-22 02:00:00,0.000000,0.0000,0.0,0.0,0,0
2022-11-22 03:00:00,0.000000,0.0000,0.0,0.0,0,0
2022-11-22 04:00:00,0.000000,0.0000,0.0,0.0,0,0
...,...,...,...,...,...,...
2022-11-29 19:00:00,0.242045,2.1625,0.0,0.0,0,1
2022-11-29 20:00:00,0.297561,0.0000,0.0,0.0,0,0
2022-11-29 21:00:00,0.000000,0.0000,0.0,0.0,0,0


### Exlude today, use up until yesterday

Only use dates excluding the date we want to predict, so use all up until "yesterday"

In [57]:
def df_yesterday_date(self, df, date):
    import pandas as pd

    yesterday = (pd.to_datetime(date) - pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    return df[:yesterday]
# add to Load agent
setattr(Load_Agent, 'df_yesterday_date', df_yesterday_date)
del df_yesterday_date 

In [58]:
Load_Agent_i = Load_Agent(df) 
Load_Agent_i.df_yesterday_date(df, date)

attributes_id,573,579,603,605,573_usage,579_usage
last_updated,,,,,,
2022-11-21 17:00:00,22.990909,0.000000,0.0,0.0,1,0
2022-11-21 18:00:00,24.871795,0.320513,0.0,0.0,1,0
2022-11-21 19:00:00,0.000000,0.000000,0.0,0.0,0,0
2022-11-21 20:00:00,14.541727,1.000719,0.0,0.0,1,0
2022-11-21 21:00:00,9.292381,0.882857,0.0,0.0,0,0
...,...,...,...,...,...,...
2022-11-29 19:00:00,0.242045,2.162500,0.0,0.0,0,1
2022-11-29 20:00:00,0.297561,0.000000,0.0,0.0,0,0
2022-11-29 21:00:00,0.000000,0.000000,0.0,0.0,0,0


### Creating df_hours
Creating raw load profile for each hour of each day
Creating new data frame df_hours

In [59]:
def load_profile_raw(self, df, shiftable_devices):
    import pandas as pd

    hours = []
    for hour in range(1, 25):
        hours.append("h" + str(hour))
    df_hours = {}

    for idx, appliance in enumerate(
        shiftable_devices
    ):
        df_hours[appliance] = pd.DataFrame(index=None, columns=hours)
        column = df[appliance]

        for i in range(len(column)):

            if (i == 0) and (column[0] > 0):
                df_hours[appliance].loc[0, "h" + str(1)] = column[0]

            elif (column[i - 1] == 0) and (column[i] > 0):
                for j in range(0, 24):
                    if (i + j) < len(column):
                        if column[i + j] > 0:
                            df_hours[appliance].loc[i, "h" + str(j + 1)] = column[
                                i + j
                            ]
    return df_hours

# add to Load agent
setattr(Load_Agent, 'load_profile_raw', load_profile_raw)
del load_profile_raw 

In [60]:
Load_Agent_i = Load_Agent(df) 
df_hours = Load_Agent_i.load_profile_raw(df, shiftable_devices)

In [61]:
df_hours

{573:             h1         h2         h3         h4        h5         h6  \
 0    22.990909        NaN        NaN        NaN       NaN        NaN   
 3    14.541727   9.292381        NaN        NaN       NaN        NaN   
 194   0.242045   0.297561        NaN        NaN       NaN        NaN   
 206      0.375   0.342424   1.407843        NaN  20.07551  26.574286   
 210   20.07551  26.574286  29.324176  26.513514       NaN        NaN   
 216       21.1        NaN        NaN        NaN       NaN        NaN   
 
             h7         h8   h9  h10  ...       h15  h16       h17        h18  \
 0          NaN        NaN  NaN  NaN  ...       NaN  NaN       NaN        NaN   
 3          NaN        NaN  NaN  NaN  ...       NaN  NaN       NaN        NaN   
 194        NaN        NaN  NaN  NaN  ...  1.407843  NaN  20.07551  26.574286   
 206  29.324176  26.513514  NaN  NaN  ...       NaN  NaN       NaN        NaN   
 210       21.1        NaN  NaN  NaN  ...       NaN  NaN       NaN        NaN

### Cleaning df_hours
Cleaning the new data frame df_hours

In [62]:
def load_profile_cleaned(self, df_hours):
    import numpy as np

    for app in df_hours.keys():
        for i in df_hours[app].index:
            for j in df_hours[app].columns:
                if np.isnan(df_hours[app].loc[i, j]):
                    df_hours[app].loc[i, j:] = 0
    return df_hours

# add to Load agent
setattr(Load_Agent, 'load_profile_cleaned', load_profile_cleaned)
del load_profile_cleaned 

In [63]:
Load_Agent_i = Load_Agent(df) 
df_hours = Load_Agent_i.load_profile_cleaned(df_hours)

In [64]:
df_hours

{573:             h1         h2         h3         h4 h5 h6 h7 h8 h9 h10  ... h15  \
 0    22.990909          0          0          0  0  0  0  0  0   0  ...   0   
 3    14.541727   9.292381          0          0  0  0  0  0  0   0  ...   0   
 194   0.242045   0.297561          0          0  0  0  0  0  0   0  ...   0   
 206      0.375   0.342424   1.407843          0  0  0  0  0  0   0  ...   0   
 210   20.07551  26.574286  29.324176  26.513514  0  0  0  0  0   0  ...   0   
 216       21.1          0          0          0  0  0  0  0  0   0  ...   0   
 
     h16 h17 h18 h19 h20 h21 h22 h23 h24  
 0     0   0   0   0   0   0   0   0   0  
 3     0   0   0   0   0   0   0   0   0  
 194   0   0   0   0   0   0   0   0   0  
 206   0   0   0   0   0   0   0   0   0  
 210   0   0   0   0   0   0   0   0   0  
 216   0   0   0   0   0   0   0   0   0  
 
 [6 rows x 24 columns],
 579:            h1        h2        h3        h4 h5 h6 h7 h8 h9 h10  ... h15 h16  \
 1    0.320513       

### Load Profiles


In [66]:
def load_profile(self, df_hours, shiftable_devices):
    import pandas as pd

    hours = df_hours[shiftable_devices[0]].columns
    loads = pd.DataFrame(columns=hours)

    for app in df_hours.keys():
        app_mean = df_hours[app].apply(lambda x: x.mean(), axis=0)
        for hour in app_mean.index:
            loads.loc[app, hour] = app_mean[hour]

    loads = loads.fillna(0)
    return loads
# add to Load agent
setattr(Load_Agent, 'load_profile', load_profile)
del load_profile 

In [67]:
Load_Agent_i = Load_Agent(df) 
loads = Load_Agent_i.load_profile(df_hours, shiftable_devices)
loads

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
573,13.220865,6.084442,5.122003,4.418919,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,2.383889,0.889143,0.799011,0.350270,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Evaluation Load

### Get true loads

In [68]:
# evaluating the performance of the load agent
# -------------------------------------------------------------------------------------------
def get_true_loads(self, shiftable_devices):
    true_loads = self.load_profile_raw(self.input, shiftable_devices)
    true_loads = self.load_profile_cleaned(true_loads)
    for device, loads in true_loads.items():
        true_loads[device].rename(
            index=dict(enumerate(self.input.index)), inplace=True
        )
    return true_loads
# add to Load agent
setattr(Load_Agent, 'get_true_loads', get_true_loads)
del get_true_loads 

In [69]:
Load_Agent_i = Load_Agent(df) 
true_loads = Load_Agent_i.get_true_loads(shiftable_devices)
true_loads

{573:                             h1         h2         h3         h4 h5 h6 h7 h8  \
 2022-11-21 17:00:00  22.990909          0          0          0  0  0  0  0   
 2022-11-21 20:00:00  14.541727   9.292381          0          0  0  0  0  0   
 2022-11-29 19:00:00   0.242045   0.297561          0          0  0  0  0  0   
 2022-11-30 07:00:00      0.375   0.342424   1.407843          0  0  0  0  0   
 2022-11-30 11:00:00   20.07551  26.574286  29.324176  26.513514  0  0  0  0   
 2022-11-30 17:00:00       21.1          0          0          0  0  0  0  0   
 
                     h9 h10  ... h15 h16 h17 h18 h19 h20 h21 h22 h23 h24  
 2022-11-21 17:00:00  0   0  ...   0   0   0   0   0   0   0   0   0   0  
 2022-11-21 20:00:00  0   0  ...   0   0   0   0   0   0   0   0   0   0  
 2022-11-29 19:00:00  0   0  ...   0   0   0   0   0   0   0   0   0   0  
 2022-11-30 07:00:00  0   0  ...   0   0   0   0   0   0   0   0   0   0  
 2022-11-30 11:00:00  0   0  ...   0   0   0   0   0   0  

### Create pipline

In [70]:
# pipeline function: creating typical load profiles
# -------------------------------------------------------------------------------------------
def pipeline(self, df, date, shiftable_devices):
    df = self.prove_start_end_date(df, date)
    df = self.df_yesterday_date(df, date)
    df_hours = self.load_profile_raw(df, shiftable_devices)
    df_hours = self.load_profile_cleaned(df_hours)
    loads = self.load_profile(df_hours, shiftable_devices)
    return loads
# add to Load agent
setattr(Load_Agent, 'pipeline', pipeline)
del pipeline 


In [71]:
Load_Agent_i = Load_Agent(df) 
loads = Load_Agent_i.pipeline(df, date, shiftable_devices)
loads

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
573,0.242045,0.297561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,2.162500,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Evaluation

In [72]:
def evaluate(self, shiftable_devices, date, metric="mse", aggregate=True, evaluation=False):
    from tqdm import tqdm
    import pandas as pd
    import numpy as np
    tqdm.pandas()

    if metric == "mse":
        import sklearn.metrics

        metric = sklearn.metrics.mean_squared_error

    true_loads = self.get_true_loads(shiftable_devices)

    scores = {}
    if not evaluation:
        for device in shiftable_devices:
            true_loads[device] = self.prove_start_end_date(true_loads[device], date)
            scores[device] = true_loads[device].progress_apply(
                lambda row: metric(
                    row.values,
                    self.pipeline(
                        self.input, str(row.name)[:10], [device]
                    ).values.reshape(
                        -1,
                    ),
                ),
                axis=1,
            )
    else:
        for device in shiftable_devices:
            true_loads[device] = self.prove_start_end_date(true_loads[device], date)
            scores[device] = {}
            for idx in tqdm(true_loads[device].index):
                date = str(idx)[:10]
                y_true = true_loads[device].loc[idx, :].values
                try:
                    y_hat = (df.loc[date][device].values.reshape(-1,))
                except KeyError:
                    try:
                        y_hat = self.pipeline(
                            self.input, date, [device]
                        ).values.reshape(
                            -1,
                        )
                    except:
                        y_hat = np.full(24, 0)
                scores[device][idx] = metric(y_true, y_hat)
            scores[device] = pd.Series(scores[device], dtype='float64')

    if aggregate:
        scores = {device: scores_df.mean() for device, scores_df in scores.items()}
    return scores

# add to Load agent
setattr(Load_Agent, 'evaluate', evaluate)
del evaluate 

In [73]:
date = '2022-11-30'

In [74]:
Load_Agent_i = Load_Agent(df) 
scores = Load_Agent_i.evaluate(shiftable_devices, date, evaluation=False)
scores

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 76.94it/s]


{573: 0.006130355677997731, 579: 0.19485026041666667}

In [75]:
Load_Agent_i = Load_Agent(df) 
scores = Load_Agent_i.evaluate(shiftable_devices, date, evaluation=True)
scores

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.01it/s]
0it [00:00, ?it/s]


{573: 0.012260711355995463, 579: nan}

# Complete Load Agent

In [76]:
# Load Agent
# ===============================================================================================
class Load_Agent:
    def __init__(self, load_input_df):
        self.input = load_input_df

    # selecting the correct data, identifying device runs, creating load profiles
    # -------------------------------------------------------------------------------------------
    def prove_start_end_date(self, df, date):
        import pandas as pd

        start_date = (df.index[0]).strftime("%Y-%m-%d")
        end_date = date

        if len(df.loc[start_date]) < 24:
            start_date = (pd.to_datetime(start_date) + pd.Timedelta(days=1)).strftime(
                "%Y-%m-%d"
            )
            df = df[start_date:end_date]
        else:
            df = df[:end_date]

        if len(df.loc[end_date]) < 24:
            end_new = (pd.to_datetime(end_date) - pd.Timedelta(days=1)).strftime(
                "%Y-%m-%d"
            )
            df = df[:end_new]
        else:
            df = df[:end_date]
        return df

    def df_yesterday_date(self, df, date):
        import pandas as pd

        yesterday = (pd.to_datetime(date) - pd.Timedelta(days=1)).strftime("%Y-%m-%d")
        return df[:yesterday]

    def load_profile_raw(self, df, shiftable_devices):
        import pandas as pd

        hours = []
        for hour in range(1, 25):
            hours.append("h" + str(hour))
        df_hours = {}

        for idx, appliance in enumerate(
            shiftable_devices
        ):
            df_hours[appliance] = pd.DataFrame(index=None, columns=hours)
            column = df[appliance]

            for i in range(len(column)):

                if (i == 0) and (column[0] > 0):
                    df_hours[appliance].loc[0, "h" + str(1)] = column[0]

                elif (column[i - 1] == 0) and (column[i] > 0):
                    for j in range(0, 24):
                        if (i + j) < len(column):
                            if column[i + j] > 0:
                                df_hours[appliance].loc[i, "h" + str(j + 1)] = column[
                                    i + j
                                ]
        return df_hours

    def load_profile_cleaned(self, df_hours):
        import numpy as np

        for app in df_hours.keys():
            for i in df_hours[app].index:
                for j in df_hours[app].columns:
                    if np.isnan(df_hours[app].loc[i, j]):
                        df_hours[app].loc[i, j:] = 0
        return df_hours

    def load_profile(self, df_hours, shiftable_devices):
        import pandas as pd

        hours = df_hours[shiftable_devices[0]].columns
        loads = pd.DataFrame(columns=hours)

        for app in df_hours.keys():
            app_mean = df_hours[app].apply(lambda x: x.mean(), axis=0)
            for hour in app_mean.index:
                loads.loc[app, hour] = app_mean[hour]

        loads = loads.fillna(0)
        return loads

    # evaluating the performance of the load agent
    # -------------------------------------------------------------------------------------------
    def get_true_loads(self, shiftable_devices):
        true_loads = self.load_profile_raw(self.input, shiftable_devices)
        true_loads = self.load_profile_cleaned(true_loads)
        for device, loads in true_loads.items():
            true_loads[device].rename(
                index=dict(enumerate(self.input.index)), inplace=True
            )
        return true_loads

    def evaluate(self, shiftable_devices, date, metric="mse", aggregate=True, evaluation=False):
        from tqdm import tqdm
        import pandas as pd
        import numpy as np
        tqdm.pandas()

        if metric == "mse":
            import sklearn.metrics

            metric = sklearn.metrics.mean_squared_error

        true_loads = self.get_true_loads(shiftable_devices)

        scores = {}
        if not evaluation:
            for device in shiftable_devices:
                true_loads[device] = self.prove_start_end_date(true_loads[device], date)
                scores[device] = true_loads[device].progress_apply(
                    lambda row: metric(
                        row.values,
                        self.pipeline(
                            self.input, str(row.name)[:10], [device]
                        ).values.reshape(
                            -1,
                        ),
                    ),
                    axis=1,
                )
        else:
            for device in shiftable_devices:
                true_loads[device] = self.prove_start_end_date(true_loads[device], date)
                scores[device] = {}
                for idx in tqdm(true_loads[device].index):
                    date = str(idx)[:10]
                    y_true = true_loads[device].loc[idx, :].values
                    try:
                        y_hat = (df.loc[date][device].values.reshape(-1,))
                    except KeyError:
                        try:
                            y_hat = self.pipeline(
                                self.input, date, [device]
                            ).values.reshape(
                                -1,
                            )
                        except:
                            y_hat = np.full(24, 0)
                    scores[device][idx] = metric(y_true, y_hat)
                scores[device] = pd.Series(scores[device], dtype='float64')

        if aggregate:
            scores = {device: scores_df.mean() for device, scores_df in scores.items()}
        return scores

    # pipeline function: creating typical load profiles
    # -------------------------------------------------------------------------------------------
    def pipeline(self, df, date, shiftable_devices):
        df = self.prove_start_end_date(df, date)
        df = self.df_yesterday_date(df, date)
        df_hours = self.load_profile_raw(df, shiftable_devices)
        df_hours = self.load_profile_cleaned(df_hours)
        loads = self.load_profile(df_hours, shiftable_devices)
        return loads


In [77]:
shiftable_devices = [573,579]
load = Load_Agent(df)

date = '2022-11-30'

output = load.pipeline(df, date, shiftable_devices)
output

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
573,0.242045,0.297561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,2.162500,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
